# Retraitement des données horaires de la qualité de l'air (format xml)

#### Nous disposons de plusieurs datasets :

- une collection de fichiers xml avec l'historique horaire de la qualité de l'air pour plusieurs région de France (1 fichier = 1 journée)
- une base de donnée B
- une base de donnée D

L'objectif est de parser les différents fichiers xml pour obtenir un tableau clair des données, pour la région qui nous intéresse (Paris)

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
#import ipdb
import datetime as dt
import calendar
import numpy as np
import ipdb

In [3]:
#chemin du fichier xml qui servira de référence pour l'algo de construction du dataset
url_xml_test = r"https://files.data.gouv.fr/lcsqa/concentrations-de-polluants-atmospheriques-reglementes/old/2019/fr-2019-e2-2018-2018-12-31-23-v.xml"

In [11]:
#Requête de tous les noms de fichier 
links=[]
years = [2018]
#years = [2018,2019,2020,2021]
for year in years:
    
    url_root = "https://files.data.gouv.fr/lcsqa/concentrations-de-polluants-atmospheriques-reglementes/old/"+str(year)+"/"
    r = requests.get(url_root)
    soup_urls = BeautifulSoup(r.content,'xml')
    soup_urls = soup_urls.find_all('a')[1:]
    for url in soup_urls:
        links.append(url_root+url.text)
    
    i=0
    while i < len(links):
    
        
        if links[i][:-6]==links[i-1][:-6]:
            
            #ipdb.set_trace()
            if "t" in links[i][-5:]:
                links.pop(i)
            elif "t" in links[i-1][-5:]:
                links.pop(i-1)
            
        i+=1
print(len(links))
links

2302


['https://files.data.gouv.fr/lcsqa/concentrations-de-polluants-atmospheriques-reglementes/old/2018/fr-2018-b-lcsqa-ineris-20190724.xml',
 'https://files.data.gouv.fr/lcsqa/concentrations-de-polluants-atmospheriques-reglementes/old/2018/fr-2018-d-lcsqa-ineris-20190918-1.xml',
 'https://files.data.gouv.fr/lcsqa/concentrations-de-polluants-atmospheriques-reglementes/old/2018/fr-2018-d-v0.xml',
 'https://files.data.gouv.fr/lcsqa/concentrations-de-polluants-atmospheriques-reglementes/old/2018/fr-2018-e2-2018-2018-09-12-04-t.xml',
 'https://files.data.gouv.fr/lcsqa/concentrations-de-polluants-atmospheriques-reglementes/old/2018/fr-2018-e2-2018-2018-09-12-05-t.xml',
 'https://files.data.gouv.fr/lcsqa/concentrations-de-polluants-atmospheriques-reglementes/old/2018/fr-2018-e2-2018-2018-09-17-14-t.xml',
 'https://files.data.gouv.fr/lcsqa/concentrations-de-polluants-atmospheriques-reglementes/old/2018/fr-2018-e2-2018-2018-09-17-15-v.xml',
 'https://files.data.gouv.fr/lcsqa/concentrations-de-pollu

# Création du tableau de vérification

In [5]:
tbl_verification = pd.DataFrame({'code verification':[1,2,3],'description':['Verified','Preliminary verified','Not verified']})
tbl_verification

,code verification,description
0,1,Verified
1,2,Preliminary verified
2,3,Not verified


# Création du tableau de validité

In [6]:
tbl_validite = pd.DataFrame({'code verification':[-99,-1,1,2,3],
                                 'description': ['Not valid due to station maintenance or calibration in the observationvalidity vocabulary',
                                                 'Not valid',
                                                 'Valid',
                                                 'Valid, but below detection limit measurement value given in the observationvalidity vocabulary',
                                                 'Valid, but below detection limit and number replaced by 0.5*detection limit in the observationvalidity vocabulary']})
tbl_validite

,code verification,description
0,-99,Not valid due to station maintenance or calibr...
1,-1,Not valid
2,1,Valid
3,2,"Valid, but below detection limit measurement v..."
4,3,"Valid, but below detection limit and number re..."


# Récupération de la liste des polluants

In [7]:
#Récupération de la liste des polluants (ils ont préalablement été transposés depuis le site dans un fichier csv)
#La liste est non exhaustive et comprend seulement les polluants qui sont mesurés dans le dataset
tbl_polluants = pd.read_csv("polluants.csv",sep=";")

tbl_polluants.head(4)

,Code,Nom,Notation
0,1,Sulphur dioxide (air),SO2
1,3,Strong acidity (air),SA
2,4,Total suspended particulates (aerosol),SPM
3,5,Particulate matter < 10 µm (aerosol),PM10


# Récupération des données sur les points de prélèvement

In [144]:
"""
filename = "D_points_prelevement.xml"
#Attention, très lent
soup_spots = BeautifulSoup(open(filename),'xml')
"""

Aperçu non exhaustif : 
    
```xml
  ...
  <gml:featureMember>
  ...
  </gml:featureMember>
  <gml:featureMember>
  ...
          <base:localId>SPO-FR01001_38</base:localId>
            ...
```
<code style="background:yellow;color:black">---> Code du point de prélèvement </code>
```xml
                   ...
                    <gn:GeographicalName>
                      ...
                      <gn:spelling>
                        <gn:SpellingOfName>
                          <gn:text>SCHILTIGHEIM</gn:text>
                        </gn:SpellingOfName>
                      </gn:spelling>
                    </gn:GeographicalName>
                  </ad:adminUnit>
                  <ad:locatorDesignator>5 rue de Madrid, Espace Européen de l'Entreprise</ad:locatorDesignator>
                  <ad:postCode>67300</ad:postCode>
               
```
<code style="background:yellow;color:black">---> Ville + Département </code>
```xml
   ...
  </gml:featureMember>

```

In [146]:
"""
all_xml_data_spots = soup_spots.find_all("gml:featureMember")
#Aperçu d'un élément (l'élément 0 correspond aux données générales du fichier donc à ne pas prendre en compte)
all_xml_data_spots[1]
"""

<gml:featureMember>
<aqd:AQD_SamplingPoint gml:id="SPO-FR01001_38">
<ef:inspireId>
<base:Identifier>
<base:localId>SPO-FR01001_38</base:localId>
<base:namespace>FR.LCSQA-INERIS.AQ</base:namespace>
<base:versionId>1</base:versionId>
</base:Identifier>
</ef:inspireId>
<ef:mediaMonitored xlink:href="http://inspire.ec.europa.eu/codeList/MediaValue/air"/>
<ef:responsibleParty>
<base2:RelatedParty>
<base2:individualName>
<gmd:LocalisedCharacterString>Julien PIECHOWSKI</gmd:LocalisedCharacterString>
</base2:individualName>
<base2:organisationName>
<gmd:LocalisedCharacterString>ATMO GRAND EST</gmd:LocalisedCharacterString>
</base2:organisationName>
<base2:contact>
<base2:Contact>
<base2:address>
<ad:AddressRepresentation>
<ad:adminUnit>
<gn:GeographicalName>
<gn:language>fra</gn:language>
<gn:nativeness nilReason="missing" xsi:nil="true"/>
<gn:nameStatus nilReason="missing" xsi:nil="true"/>
<gn:sourceOfName nilReason="missing" xsi:nil="true"/>
<gn:pronunciation nilReason="missing" xsi:nil="tru

In [147]:
"""
codes_zone = []
departements = []
villes = [] 

for data_spots in all_xml_data_spots[1:]:
    try:
        code_zone = data_spots.find_all('base:localId')[0].text.split('-')[1].split('_')[0]
        departement = int(data_spots.find_all('ad:postCode')[0].text)
        ville = data_spots.find_all('gn:SpellingOfName')[0].find_all('gn:text')[0].text
    except:
        pass
    if not departement in departements:
        codes_zone.append(code_zone)
        departements.append(departement)
        villes.append(ville)
    
tbl_zones_prelev = pd.DataFrame({'code_zone':codes_zone,'departement':departements,'ville':villes})
tbl_zones_prelev.sort_values(['departement'],inplace=True)
tbl_zones_prelev
"""

,code_zone,departement,ville
1,FR02001,13006,MARSEILLE
16,FR41001,20250,CORTE
10,FR26001,25000,BESANCON
6,FR08003,31300,TOULOUSE
7,FR09003,33692,MERIGNACÂ CEDEX
8,FR19002,35200,RENNES
9,FR23001,44307,NANTES CEDEX 3
11,FR34003,45590,SAINT-CYR-EN-VAL
4,FR06001,59800,LILLE
0,FR01001,67300,SCHILTIGHEIM


Le dataset des villes n'est pas assez complet. Dans les csv de ce dossier : https://files.data.gouv.fr/lcsqa/concentrations-de-polluants-atmospheriques-reglementes/temps-reel/2021/, les codes pour paris sont : 

In [8]:
dict_codes_paris = {'PARIS1':'FR04055' ,'PARIS7': 'FR04060', 'PARIS12': 'FR04014','PARIS13':'FR04037' ,'PARIS18':'FR04004'}
dict_codes_paris

{'PARIS1': 'FR04055',
 'PARIS7': 'FR04060',
 'PARIS12': 'FR04014',
 'PARIS13': 'FR04037',
 'PARIS18': 'FR04004'}

# Récupération et retraitement des data xml sur la qualité de l'air
- https://files.data.gouv.fr/lcsqa/concentrations-de-polluants-atmospheriques-reglementes/old/2019/
- https://files.data.gouv.fr/lcsqa/concentrations-de-polluants-atmospheriques-reglementes/old/2020/
- https://files.data.gouv.fr/lcsqa/concentrations-de-polluants-atmospheriques-reglementes/old/2021/

#### Notes sur les différentes balises
-----------------------------------------------
```xml
<gml:TimePeriod gml:id="REP_TP_FR_2018-12-31-23_E2">
<gml:beginPosition>2018-09-19T00:00:00Z</gml:beginPosition>
<gml:endPosition>2018-10-10T00:00:00Z</gml:endPosition>
</gml:TimePeriod>
```
Période sur laquelle s'étend la donnée dans tout le fichier.

=================================================================================================

```xml
<aqd:content xlink:href="FR.LCSQA-INERIS.AQ/FR_OBS_et5tb64ner_FR070A_FR24040_7_1"/>
<aqd:content xlink:href="FR.LCSQA-INERIS.AQ/FR_OBS_86q294dv4i_FR070A_FR24039_5_2"/>
...
```
Points de prélèvement et polluants pour lesquelles les données sont fournies.

FR070A_FR24040_7 : [code reseau mesure]_ [code station]_ [n° polluant]

Numéros des polluants : http://dd.eionet.europa.eu/vocabulary/aq/pollutant/view?page=1#vocabularyConceptResults

=================================================================================================
```xml
<om:parameter>
<om:NamedValue>
<om:name xlink:href="http://dd.eionet.europa.eu/vocabulary/aq/processparameter/SamplingPoint"/>
<om:value xlink:href="FR.LCSQA-INERIS.AQ/SPO-FR24040_7"/> 
```
<code style="background:yellow;color:black">--- FR24040 : point de prélèvement</code>
```xml
</om:NamedValue>
</om:parameter>
<om:observedProperty xlink:href="http://dd.eionet.europa.eu/vocabulary/aq/pollutant/7"/> 
``` 
<code style="background:yellow;color:black">---> 7 : numéro de polluant</code>
```xml
<om:featureOfInterest xlink:href="FR.LCSQA-INERIS.AQ/SAM-FR24040_7"/>
<om:result>
<swe:DataArray>
<swe:elementCount>
<swe:Count>
<swe:value>24</swe:value>
</swe:Count>
</swe:elementCount>
``` 
<code style="background:yellow;color:black">---> 24 : nombre de données horaires disponibles sur la période de mesure</code>
```xml

<swe:elementType name="FixedObservations">
<swe:encoding>
<swe:TextEncoding blockSeparator="@@" decimalSeparator="." tokenSeparator=","/>
</swe:encoding>
<swe:values>2018-10-09T00:00:00+00:00,2018-10-09T01:00:00+00:00,2,1,61.0@@2018-10-09T01:00:00+00:00,2018-10-09T02:00:00+00:00,2,1,67.7@@2018-10-09T02:00:00+00:00,2018-10-09T03:00:00+00:00,2,1,65.0@@2018-10-09T03:00:00+00:00,2018-10-09T04:00:00+00:00,2,1,62.8@@2018-10-09T04:00:00+00:00,2018-10-09T05:00:00+00:00,2,1,66.5@@2018-10-09T05:00:00+00:00,2018-10-09T06:00:00+00:00,2,1,66.0@@</swe:values>
 ``` 
<code style="background:yellow;color:black">---> Groupe de valeurs séparées par "@@" </code>
    
<code style="background:yellow;color:black">2018-10-09T00:00:00+00:00,2018-10-09T01:00:00+00:00,2,1,61.0 = StartTime,EndTime,Verification (1), Validity (2), Value </code>

(1) voir tbl_verification


(2) voir tbl_validite

In [150]:
#Requête sur le code xml
#r = requests.get(url_xml_test)
#soup_data = BeautifulSoup(r.content,'xml')

Aperçu du résultat (non exhaustif) : 
    
```xml
...
<gml:featureMember>
...
</gml:featureMember>
<gml:featureMember>
...
<om:name xlink:href="http://dd.eionet.europa.eu/vocabulary/aq/processparameter/SamplingPoint"/>
<om:value xlink:href="FR.LCSQA-INERIS.AQ/SPO-FR24040_7"/>
...
<swe:Count>
<swe:value>24</swe:value>
</swe:Count>
...
<swe:encoding>
<swe:TextEncoding blockSeparator="@@" decimalSeparator="." tokenSeparator=","/>
</swe:encoding>
<swe:values>2018-10-09T00:00:00+00:00,2018-10-09T01:00:00+00:00,2,1,79.7@@2018-10-09T01:00:00+00:00,2018-10-09T02:00:00+00:00,2,1,85.7@@2018-10-09T02:00:00+00:00,2018-10-09T03:00:00+00:00,2,1,77.8@@2018-10-09T03:00:00+00:00,2018-10-09T04:00:00+00:00,2,1,80.8@@2018-10-09T04:00:00+00:00,2018-10-09T05:00:00+00:00,2,1,77.8@@2018-10-09T05:00:00+00:00,2018-10-09T06:00:00+00:00,2,1,73.3@@</swe:values>
...
</gml:featureMember>
```

In [151]:
#Séparation des différentes données
#Renvoie une liste dont l'élément 0 correspond aux infos globales relative au fichier. 
#Nous nous intéressons aux indices [1:-1].





#all_xml_data

In [12]:
start_time = dt.datetime.now()
dates = []
h_debut = []
h_fin = []
codes_prelev = []
verifs = []
valid = []
codes_pollu = []
valeurs = []
loop=0
for url in links:
    loop+=1
    print(str(loop) + ' : working on ' + url[100:])
    
    r = requests.get(url_xml_test)
    soup_data = BeautifulSoup(r.content,'xml')
    
    all_xml_data = soup_data.find_all("gml:featureMember")
    
    #Optimisation : on vérifie en amont si il y a le code paris dans notre soupe
    paris_in_soup = False
    
    str_soup = str(soup_data)
    for code_paris in dict_codes_paris.values():
        if code_paris in str_soup:
            paris_in_soup = True
            break
            
    if paris_in_soup:
        print ("Paris in file")
        
        for data in all_xml_data[1:]:
            
            #Récupération du code du point de prélèvement et du polluant 
            ref = str(data.find_all("om:value")[1])
            lst = ref.split('-')[-1].split("\"")[0].split("_")

            code_zone_prelev = lst[0]

            if code_zone_prelev in dict_codes_paris.values():
                code_polluant = lst[1]


                #if code_zone_prelev in dict_codes_paris.values():
                #Récupération des données
                values = data.find_all("swe:values")[0].text
                values_split = values.split('@@')[:-2]

                for elem in values_split:
                    elem = elem.split(",")

                    dates.append(elem[0][:9])
                    h_debut.append(elem[0])
                    h_fin.append(elem[1])
                    codes_prelev.append(code_zone_prelev)
                    verifs.append(elem[2])
                    valid.append(elem[3])
                    codes_pollu.append(code_polluant)
                    valeurs.append(elem[4])


tbl_dataset = pd.DataFrame({"date":dates,
                           "h début":h_debut,
                            "h fin":h_fin,
                            "codes_prelev":codes_prelev,
                            "verif":verifs,
                            "valid":valid,
                            "code_polluant":codes_pollu,
                            "valeur":valeurs,
                           })
    



print("time", (dt.datetime.now() - start_time))
tbl_dataset

1 : working on 2018-b-lcsqa-ineris-20190724.xml
2 : working on 2018-d-lcsqa-ineris-20190918-1.xml
3 : working on 2018-d-v0.xml
4 : working on 2018-e2-2018-2018-09-12-04-t.xml
5 : working on 2018-e2-2018-2018-09-12-05-t.xml
6 : working on 2018-e2-2018-2018-09-17-14-t.xml
7 : working on 2018-e2-2018-2018-09-17-15-v.xml
8 : working on 2018-e2-2018-2018-09-17-16-v.xml
9 : working on 2018-e2-2018-2018-09-17-17-t.xml
10 : working on 2018-e2-2018-2018-09-17-18-v.xml
11 : working on 2018-e2-2018-2018-09-17-19-v.xml
12 : working on 2018-e2-2018-2018-09-17-22-v.xml
13 : working on 2018-e2-2018-2018-09-17-23-v.xml
14 : working on 2018-e2-2018-2018-09-18-00-v.xml
15 : working on 2018-e2-2018-2018-09-18-01-v.xml
16 : working on 2018-e2-2018-2018-09-18-02-v.xml
17 : working on 2018-e2-2018-2018-09-18-03-v.xml
18 : working on 2018-e2-2018-2018-09-18-04-v.xml
19 : working on 2018-e2-2018-2018-09-18-05-v.xml
20 : working on 2018-e2-2018-2018-09-18-06-v.xml
21 : working on 2018-e2-2018-2018-09-18-07-t.x

168 : working on 2018-e2-2018-2018-09-25-17-t.xml
169 : working on 2018-e2-2018-2018-09-25-18-v.xml
170 : working on 2018-e2-2018-2018-09-25-19-v.xml
171 : working on 2018-e2-2018-2018-09-25-20-v.xml
172 : working on 2018-e2-2018-2018-09-25-21-v.xml
173 : working on 2018-e2-2018-2018-09-25-22-v.xml
174 : working on 2018-e2-2018-2018-09-25-23-t.xml
175 : working on 2018-e2-2018-2018-09-26-04-t.xml
176 : working on 2018-e2-2018-2018-09-26-05-t.xml
177 : working on 2018-e2-2018-2018-09-26-06-t.xml
178 : working on 2018-e2-2018-2018-09-26-07-t.xml
179 : working on 2018-e2-2018-2018-09-26-08-t.xml
180 : working on 2018-e2-2018-2018-09-26-09-t.xml
181 : working on 2018-e2-2018-2018-09-26-10-t.xml
182 : working on 2018-e2-2018-2018-09-26-11-t.xml
183 : working on 2018-e2-2018-2018-09-26-12-t.xml
184 : working on 2018-e2-2018-2018-09-26-13-t.xml
185 : working on 2018-e2-2018-2018-09-26-14-t.xml
186 : working on 2018-e2-2018-2018-09-26-15-v.xml
187 : working on 2018-e2-2018-2018-09-26-16-v.xml


332 : working on 2018-e2-2018-2018-10-04-08-t.xml
333 : working on 2018-e2-2018-2018-10-04-09-t.xml
334 : working on 2018-e2-2018-2018-10-04-10-t.xml
335 : working on 2018-e2-2018-2018-10-04-11-t.xml
336 : working on 2018-e2-2018-2018-10-04-12-t.xml
337 : working on 2018-e2-2018-2018-10-04-13-t.xml
338 : working on 2018-e2-2018-2018-10-04-14-t.xml
339 : working on 2018-e2-2018-2018-10-04-15-v.xml
340 : working on 2018-e2-2018-2018-10-04-16-v.xml
341 : working on 2018-e2-2018-2018-10-04-17-t.xml
342 : working on 2018-e2-2018-2018-10-04-18-v.xml
343 : working on 2018-e2-2018-2018-10-04-19-v.xml
344 : working on 2018-e2-2018-2018-10-04-20-v.xml
345 : working on 2018-e2-2018-2018-10-04-21-v.xml
346 : working on 2018-e2-2018-2018-10-04-22-v.xml
347 : working on 2018-e2-2018-2018-10-05-04-t.xml
348 : working on 2018-e2-2018-2018-10-05-05-t.xml
349 : working on 2018-e2-2018-2018-10-05-06-t.xml
350 : working on 2018-e2-2018-2018-10-05-07-t.xml
351 : working on 2018-e2-2018-2018-10-05-08-t.xml


496 : working on 2018-e2-2018-2018-10-12-18-v.xml
497 : working on 2018-e2-2018-2018-10-12-19-v.xml
498 : working on 2018-e2-2018-2018-10-12-20-v.xml
499 : working on 2018-e2-2018-2018-10-12-21-v.xml
500 : working on 2018-e2-2018-2018-10-12-22-v.xml
501 : working on 2018-e2-2018-2018-10-12-23-v.xml
502 : working on 2018-e2-2018-2018-10-13-04-t.xml
503 : working on 2018-e2-2018-2018-10-13-05-t.xml
504 : working on 2018-e2-2018-2018-10-13-06-t.xml
505 : working on 2018-e2-2018-2018-10-13-07-t.xml
506 : working on 2018-e2-2018-2018-10-13-08-t.xml
507 : working on 2018-e2-2018-2018-10-13-09-t.xml
508 : working on 2018-e2-2018-2018-10-13-10-t.xml
509 : working on 2018-e2-2018-2018-10-13-11-t.xml
510 : working on 2018-e2-2018-2018-10-13-12-t.xml
511 : working on 2018-e2-2018-2018-10-13-13-t.xml
512 : working on 2018-e2-2018-2018-10-13-14-t.xml
513 : working on 2018-e2-2018-2018-10-13-15-v.xml
514 : working on 2018-e2-2018-2018-10-13-16-v.xml
515 : working on 2018-e2-2018-2018-10-13-17-t.xml


660 : working on 2018-e2-2018-2018-10-20-18-v.xml
661 : working on 2018-e2-2018-2018-10-20-19-v.xml
662 : working on 2018-e2-2018-2018-10-20-20-v.xml
663 : working on 2018-e2-2018-2018-10-20-21-v.xml
664 : working on 2018-e2-2018-2018-10-20-22-t.xml
665 : working on 2018-e2-2018-2018-10-21-04-t.xml
666 : working on 2018-e2-2018-2018-10-21-05-t.xml
667 : working on 2018-e2-2018-2018-10-21-06-t.xml
668 : working on 2018-e2-2018-2018-10-21-07-t.xml
669 : working on 2018-e2-2018-2018-10-21-08-t.xml
670 : working on 2018-e2-2018-2018-10-21-09-t.xml
671 : working on 2018-e2-2018-2018-10-21-10-t.xml
672 : working on 2018-e2-2018-2018-10-21-11-t.xml
673 : working on 2018-e2-2018-2018-10-21-12-t.xml
674 : working on 2018-e2-2018-2018-10-21-13-t.xml
675 : working on 2018-e2-2018-2018-10-21-14-t.xml
676 : working on 2018-e2-2018-2018-10-21-15-v.xml
677 : working on 2018-e2-2018-2018-10-21-16-v.xml
678 : working on 2018-e2-2018-2018-10-21-17-t.xml
679 : working on 2018-e2-2018-2018-10-21-18-v.xml


...

1793 : working on 2018-e2-2018-2018-12-07-23-v.xml
1794 : working on 2018-e2-2018-2018-12-08-00-v.xml
1795 : working on 2018-e2-2018-2018-12-08-01-t.xml
1796 : working on 2018-e2-2018-2018-12-08-02-t.xml
1797 : working on 2018-e2-2018-2018-12-08-03-t.xml
1798 : working on 2018-e2-2018-2018-12-08-04-t.xml
1799 : working on 2018-e2-2018-2018-12-08-05-t.xml
1800 : working on 2018-e2-2018-2018-12-08-06-t.xml
1801 : working on 2018-e2-2018-2018-12-08-07-t.xml
1802 : working on 2018-e2-2018-2018-12-08-08-t.xml
1803 : working on 2018-e2-2018-2018-12-08-09-t.xml
1804 : working on 2018-e2-2018-2018-12-08-10-t.xml
1805 : working on 2018-e2-2018-2018-12-08-11-t.xml
1806 : working on 2018-e2-2018-2018-12-08-12-t.xml
1807 : working on 2018-e2-2018-2018-12-08-13-t.xml
1808 : working on 2018-e2-2018-2018-12-08-14-t.xml
1809 : working on 2018-e2-2018-2018-12-08-15-v.xml
1810 : working on 2018-e2-2018-2018-12-08-16-v.xml
1811 : working on 2018-e2-2018-2018-12-08-17-t.xml
1812 : working on 2018-e2-2018-

1954 : working on 2018-e2-2018-2018-12-14-21-v.xml
1955 : working on 2018-e2-2018-2018-12-14-22-v.xml
1956 : working on 2018-e2-2018-2018-12-14-23-v.xml
1957 : working on 2018-e2-2018-2018-12-15-00-t.xml
1958 : working on 2018-e2-2018-2018-12-15-01-t.xml
1959 : working on 2018-e2-2018-2018-12-15-02-t.xml
1960 : working on 2018-e2-2018-2018-12-15-03-t.xml
1961 : working on 2018-e2-2018-2018-12-15-04-t.xml
1962 : working on 2018-e2-2018-2018-12-15-05-t.xml
1963 : working on 2018-e2-2018-2018-12-15-06-t.xml
1964 : working on 2018-e2-2018-2018-12-15-07-t.xml
1965 : working on 2018-e2-2018-2018-12-15-08-t.xml
1966 : working on 2018-e2-2018-2018-12-15-09-t.xml
1967 : working on 2018-e2-2018-2018-12-15-10-t.xml
1968 : working on 2018-e2-2018-2018-12-15-11-t.xml
1969 : working on 2018-e2-2018-2018-12-15-12-t.xml
1970 : working on 2018-e2-2018-2018-12-15-13-t.xml
1971 : working on 2018-e2-2018-2018-12-15-14-t.xml
1972 : working on 2018-e2-2018-2018-12-15-15-v.xml
1973 : working on 2018-e2-2018-

2115 : working on 2018-e2-2018-2018-12-21-14-t.xml
2116 : working on 2018-e2-2018-2018-12-21-15-v.xml
2117 : working on 2018-e2-2018-2018-12-21-16-v.xml
2118 : working on 2018-e2-2018-2018-12-21-17-t.xml
2119 : working on 2018-e2-2018-2018-12-21-18-t.xml
2120 : working on 2018-e2-2018-2018-12-21-19-v.xml
2121 : working on 2018-e2-2018-2018-12-21-20-v.xml
2122 : working on 2018-e2-2018-2018-12-21-21-v.xml
2123 : working on 2018-e2-2018-2018-12-21-22-v.xml
2124 : working on 2018-e2-2018-2018-12-21-23-v.xml
2125 : working on 2018-e2-2018-2018-12-22-00-t.xml
2126 : working on 2018-e2-2018-2018-12-22-01-t.xml
2127 : working on 2018-e2-2018-2018-12-22-02-t.xml
2128 : working on 2018-e2-2018-2018-12-22-03-t.xml
2129 : working on 2018-e2-2018-2018-12-22-04-t.xml
2130 : working on 2018-e2-2018-2018-12-22-05-t.xml
2131 : working on 2018-e2-2018-2018-12-22-06-t.xml
2132 : working on 2018-e2-2018-2018-12-22-07-t.xml
2133 : working on 2018-e2-2018-2018-12-22-08-t.xml
2134 : working on 2018-e2-2018-

2276 : working on 2018-e2-2018-2018-12-30-03-t.xml
2277 : working on 2018-e2-2018-2018-12-30-04-t.xml
2278 : working on 2018-e2-2018-2018-12-30-05-t.xml
2279 : working on 2018-e2-2018-2018-12-30-06-t.xml
2280 : working on 2018-e2-2018-2018-12-30-07-t.xml
2281 : working on 2018-e2-2018-2018-12-30-08-t.xml
2282 : working on 2018-e2-2018-2018-12-30-09-t.xml
2283 : working on 2018-e2-2018-2018-12-30-15-v.xml
2284 : working on 2018-e2-2018-2018-12-30-16-v.xml
2285 : working on 2018-e2-2018-2018-12-30-19-v.xml
2286 : working on 2018-e2-2018-2018-12-31-00-t.xml
2287 : working on 2018-e2-2018-2018-12-31-01-t.xml
2288 : working on 2018-e2-2018-2018-12-31-02-t.xml
2289 : working on 2018-e2-2018-2018-12-31-03-t.xml
2290 : working on 2018-e2-2018-2018-12-31-04-t.xml
2291 : working on 2018-e2-2018-2018-12-31-05-t.xml
2292 : working on 2018-e2-2018-2018-12-31-06-t.xml
2293 : working on 2018-e2-2018-2018-12-31-07-t.xml
2294 : working on 2018-e2-2018-2018-12-31-08-t.xml
2295 : working on 2018-e2-2018-

,date,h début,h fin,codes_prelev,verif,valid,code_polluant,valeur
